# **`gos`** by example

This notebook demonstrates the core features of `gos`:

- Author declarative genomics visualizations which adhere to the [Gosling](http://gosling-lang.org/) JSON Specification 
- Render visualizations directly in notebook
- Combine local, remote, and in-memory genomics data sources in visualizations
- Control a Gosling visualization from Python

Start by importing `gosling`. It is a convention to import as `gos` and then access the API through this namespace. 

In [1]:
import gosling as gos

## Creating a `gos.Track`

**`gos`** exposes two fundemental building-blocks for genomics visualizatinos provided by the Gosling grammar:

- `gos.Track`
- `gos.View`

A _Track_ is the core component of a genomics visualization that defines explict **transformations** and **mappings** of genomics data to **visual properties**. A _Track_ ay be composed with other _Tracks_ or **grouped** into a _View_ that share the same linked genomic domain.

Every _Track_ therefore binds a data source. In `gos` we define an abstract genomic data source and bind it to a _Track_. We will start by loading a CSV containing UCSC hg38 cytoband information

In [11]:
csv_url = "https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.csv"

# define the dataset
data = gos.csv(
    url=csv_url,
    chromosomeField="Chromosome",
    genomicFields=["chromStart", "chromEnd"],
)

# bind the data to a track
gos.Track(data)

Track({
  data: {'type': 'csv', 'url': 'https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.csv', 'chromosomeField': 'Chromosome', 'genomicFields': ['chromStart', 'chromEnd']},
  height: 180,
  mark: 'bar',
  width: 800
})

The _Track_ above is bound to the genomics data, but we haven't declared how to map and transform these data to visual properties. For this, we will use the `gos.Track.mark_*()` and `gos.Track.encode()` method to declare which visual encoding to use and how to map and transform these data to this encoding. 

In [12]:
track = gos.Track(data).mark_rect().encode(
    # defines start and end of rectangle mark
    x=gos.X("chromStart:G", axis="top"),
    xe=gos.Xe("chromEnd:G"),
    # defines how to map values in "Stain" column to colors
    color=gos.Color(
        "Stain:N", 
        domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
        range=["white", "#D9D9D9", "#979797", "#636363", "black", "#A0A0F2"]
    ),
    # customize the style of the visual marks. 
    size=gos.value(20),
    stroke=gos.value("gray"),
    strokeWidth=gos.value(0.5)
)
track

Track({
  color: Color({
    domain: ['gneg', 'gpos25', 'gpos50', 'gpos75', 'gpos100', 'gvar'],
    range: ['white', '#D9D9D9', '#979797', '#636363', 'black', '#A0A0F2'],
    shorthand: 'Stain:N'
  }),
  data: {'type': 'csv', 'url': 'https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.csv', 'chromosomeField': 'Chromosome', 'genomicFields': ['chromStart', 'chromEnd']},
  height: 180,
  mark: 'rect',
  size: SizeValue({
    value: 20
  }),
  stroke: StrokeValue({
    value: 'gray'
  }),
  strokeWidth: StrokeWidthValue({
    value: 0.5
  }),
  width: 800,
  x: X({
    axis: 'top',
    shorthand: 'chromStart:G'
  }),
  xe: Xe({
    shorthand: 'chromEnd:G'
  })
})

Our `gos.Track` now is fully specified, however, the Gosling grammar defines the root of every visualization as a _View_, which may contain one or more _Tracks_. Therefore in order to complete a Gosling specification for the track in isolation, we use the `gos.Track.view()` method to cast the track within a `gos.View`. In Jupyter or Google Colab, the visualization will automatically be rendered in the cell below rather than printing a Python object like above.

In [13]:
track.view() # visualization rendered belows

View({
  tracks: [Track({
    color: Color({
      domain: ['gneg', 'gpos25', 'gpos50', 'gpos75', 'gpos100', 'gvar'],
      field: 'Stain',
      range: ['white', '#D9D9D9', '#979797', '#636363', 'black', '#A0A0F2'],
      type: 'nominal'
    }),
    data: {'type': 'csv', 'url': 'https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.csv', 'chromosomeField': 'Chromosome', 'genomicFields': ['chromStart', 'chromEnd']},
    height: 180,
    mark: 'rect',
    size: SizeValue({
      value: 20
    }),
    stroke: StrokeValue({
      value: 'gray'
    }),
    strokeWidth: StrokeWidthValue({
      value: 0.5
    }),
    width: 800,
    x: X({
      axis: 'top',
      field: 'chromStart',
      type: 'genomic'
    }),
    xe: Xe({
      field: 'chromEnd',
      type: 'genomic'
    })
  })]
})

Additional parameters for the `gos.View` can be passed in as well for convenience. We can easily set a `title` and `xDomain` for our _View_, initializing the visualization to display "chr1". 

> Notice how we reuse the `track` instance to create new, modified views. This is a common pattern in **`gos`**.

In [14]:
track.view(
    title="Gos is awesome!",
    xDomain=gos.GenomicDomain(chromosome="chr1"),
)

View({
  title: 'Gos is awesome!',
  tracks: [Track({
    color: Color({
      domain: ['gneg', 'gpos25', 'gpos50', 'gpos75', 'gpos100', 'gvar'],
      field: 'Stain',
      range: ['white', '#D9D9D9', '#979797', '#636363', 'black', '#A0A0F2'],
      type: 'nominal'
    }),
    data: {'type': 'csv', 'url': 'https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.csv', 'chromosomeField': 'Chromosome', 'genomicFields': ['chromStart', 'chromEnd']},
    height: 180,
    mark: 'rect',
    size: SizeValue({
      value: 20
    }),
    stroke: StrokeValue({
      value: 'gray'
    }),
    strokeWidth: StrokeWidthValue({
      value: 0.5
    }),
    width: 800,
    x: X({
      axis: 'top',
      field: 'chromStart',
      type: 'genomic'
    }),
    xe: Xe({
      field: 'chromEnd',
      type: 'genomic'
    })
  })],
  xDomain: GenomicDomain({
    chromosome: 'chr1'
  })
})